In [15]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents.agent_toolkits import AzureCognitiveServicesToolkit
toolkit = AzureCognitiveServicesToolkit()
toolkit.get_tools()

[AzureCogsFormRecognizerTool(doc_analysis_client=<azure.ai.formrecognizer._document_analysis_client.DocumentAnalysisClient object at 0x0000022FFA91BBE0>),
 AzureCogsSpeech2TextTool(speech_config=<azure.cognitiveservices.speech.SpeechConfig object at 0x0000022FD85331F0>),
 AzureCogsText2SpeechTool(speech_config=<azure.cognitiveservices.speech.SpeechConfig object at 0x0000022FD8533100>),
 AzureCogsTextAnalyticsHealthTool(text_analytics_client=<azure.ai.textanalytics._text_analytics_client.TextAnalyticsClient object at 0x0000022FFAF583A0>)]

In [3]:
from langchain.agents.agent_toolkits import AzureCognitiveServicesToolkit
toolkit = AzureCognitiveServicesToolkit()


from langchain.agents import initialize_agent, AgentType
from langchain import OpenAI
from langchain_openai import ChatOpenAI

llm = OpenAI()
Model = ChatOpenAI()

agent = initialize_agent(
    tools = toolkit.get_tools(),
    llm = llm,
    agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)


description = agent.run("what shows the following image?:"
                        "https://www.stylo24.it/wp-content/uploads/2020/03/fionda.jpg")

print(description)

C:\Users\fuksy\AppData\Local\Temp\ipykernel_19252\1845237682.py:9: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI()
C:\Users\fuksy\AppData\Local\Temp\ipykernel_19252\1845237682.py:12: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(
C:\Users\fuksy\AppData\Local\Temp\ipykernel_19252\1845237682.py:20: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  descriptio



> Entering new AgentExecutor chain...


APIConnectionError: Connection error.

In [ ]:
description = agent.run("tell me a story aloud regarding the picture?:"
                        "https://www.stylo24.it/wp-content/uploads/2020/03/fionda.jpg")



In [ ]:
from IPython import display

display.display(description)

In [ ]:
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

In [ ]:
prefix = """

        you are a story reader for children.
        you read aloud stories based on pictures you are shown.
        you build an engaging story using accentuations and pronunciations related to the story.
        you can youse multiple tools for your question.
        always use tools.
        you have access to the following tools:"""
        
agent = initialize_agent(toolkit.get_tools(), llm, agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose = True, agent_kwargs={"prefix": prefix})


description = agent.run("tell me a story aloud regarding the picture?:"
                        "https://www.stylo24.it/wp-content/uploads/2020/03/fionda.jpg")


In [ ]:
toolkit = AzureCognitiveServicesToolkit().get_tools()
tools = [toolkit[0], toolkit[2]]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)



print(agent.agent.llm_chain.prompt.messages[0].prompt.template)



In [ ]:
output = agent.run("what are all men's skus?"
    "https://www.whiteelysee.fr/design/wp-content/uploads/2022/01/custom-t-shirt-order-form-template-free.jpg")


# 2. Combining single tools into one agent

In [161]:
from langchain.agents import load_tools
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent
from langchain_openai import ChatOpenAI
from langchain.tools import tool

from openai import OpenAI as OpenAIClient
import openai

from PIL import Image
import requests
from io import BytesIO


@tool
def yt_transcribe(path):
    """
    description: Given a path from a video, this fuction runs a Speech-to-text and will transcribe all audio from the video and return the text

    Args:
        video_url (str): file.mp4

    Returns:
        str: transcription
    """
    print(f"transcribing {path}")
    audio_file = open(path.strip("'\""), 'rb')
    result = openai.audio.transcriptions.create(model = "whisper-1", file = audio_file)
    audio_file.close()
    return (result)


@tool
def create_image(topic:str):
    
    """
        description: Generate an image using DALL-E and return its URL
        
        Args:
            prompt (str): Description of the image to generate
            size (str): Image size (1024x1024, 1024x1792, or 1792x1024 for DALL-E 3)
            model (str): DALL-E model version to use
            
        Returns:
            image
    """
    client = OpenAIClient()
    model="dall-e-3"
    size="1024x1024"
    quality = 'standard'
    n = 1

    prompt_dalle = PromptTemplate(
        input_variables = ['image_description'],
        template="create a detailed image through dalle depicting the following scene: {image_description}"
    )



    response = client.images.generate(
        model=model,
        size=size,
        quality = quality,
        n = n,
        prompt = prompt_dalle.format(image_description = topic))
    image_url = response.data[0].url

    # Download and return the image
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content))
    image.save('generated_image.png')
    return {'image':image, 'url':image_url}

@tool
def files_available(file:str  = None):

    """
    description: List all files available at the current location

    Returns:
        list: List of file names
    """
    return [os.listdir()]

@tool
def remove_badcontent(texto:str):
    """
    description: remove and replace all bad words in order to make it compliance with rules of people underage

    Args:
        texto (str): texto to be cleaned
    """
    
    from langchain_openai import OpenAI
    llm_ = OpenAI()
    template = """ the text is in portuguese, make sure to clean all the bad words and possible ofenses replacing it to friendly language:
            text: {texto}
            show the replaced words
            """
    prompt_template = PromptTemplate(input_variables=['texto'],
                                    template = template)

    response = llm_.invoke(prompt_template.invoke(texto))
    return response

In [162]:
from langchain.tools import Tool
tools = [Tool(
    name = 'transcribe video',
    func = yt_transcribe, 
    description=' Given a path from a video, this fuction runs a Speech-to-text and will transcribe all audio from the video and return the text'),
    
    Tool(name = 'image generator',
    func = create_image,
    description='Generate an image using DALL-E and return its URL'),
    
    Tool(name = 'files mapping',
         func = files_available,
         description='map all files available at the current location'),
    
    Tool(name = 'remover of bad content',
         func = remove_badcontent,
         description='removes bad content making the text friendly to guidelines')
    ]

llm = ChatOpenAI()
agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION
verbose = True

agent = initialize_agent(tools, llm, agent = agent, verbose = verbose)

In [164]:
template = """you have the following tools available: 
            transcribe video: transcribes a mp4 video using whispers from openai
            image generator: builds an image using dalle from a topic informed
            files mapping: it will map all available files at the current location

            from a path where it contains a mp4 video, you should:
                1. check if the requested file is in the current folder location: Return true or false
                2. transcribe the video: return a text
                3. clean the transcription to omit and exclude offensive words and phrasing: return a text
                4. summarize it using your own llm capabilities the cleaned transcription into a single phrase
                5. generate the image from this scene: return a image
                
            for each step you take, explain what are your actions

            the file is located at:{path}
            
            your final observation should be printing the image

            """

pt = PromptTemplate(input_variables= ['path'], template = template)

result = agent.run(pt.invoke("videoplayback.mp4"))



> Entering new AgentExecutor chain...
I should first check if the requested file is in the current folder location, then transcribe the video, clean the transcription, summarize it, generate the image, and finally print the image.
Action: files mapping
Action Input: 'videoplayback.mp4'
Observation: [['.env', '.git', '.gitignore', '10_multimodal.ipynb', '6_building_conversational_apps.ipynb', '7_recomendation_systems.ipynb', '8_structured.ipynb', 'chinook.db', 'Chinook_Sqlite.sql', 'data', 'generated_image.png', 'globebotter.py', 'italy_travel.pdf', 'movies.pkl', 'movies_metadata.csv', 'Orçamento -  EDIFICIO PAULISTA NEW STYLE.pdf', 'outputs.txt', 'README.md', 'requirements.txt', 'videoplayback.mp4']]
Thought:Now that I have confirmed the file exists in the current folder, I should transcribe the video.
Action: transcribe video
Action Input: 'videoplayback.mp4'transcribing 'videoplayback.mp4'

Observation: Transcription(text='Não, abdômen tá legal. Muita testa. É verdade. Isso aqui eu

'An image has been generated from the summarized text.'